In [51]:
import pandas as pd

In [52]:
df = pd.read_csv('herg_compound_from_chembl.csv', index_col=[0])
df.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,bao_label,canonical_smiles,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,NaN,224312,[],CHEMBL824627,Compound was evaluated for the displacement of...,B,BAO_0000192,BAO_0000219,cell-based format,CC(C(=O)N[C@]1(c2ccccc2)CC[C@@H](N2CCN(C(C)C)C...,...,Homo sapiens,HERG,9606,NaN,NaN,Ki,nM,UO_0000065,NaN,1200.0
1,NaN,225577,[],CHEMBL824627,Compound was evaluated for the displacement of...,B,BAO_0000192,BAO_0000219,cell-based format,CC(C(=O)NC1(c2ccccc2)CCC(N2CCC3(CCCO3)CC2)CC1)...,...,Homo sapiens,HERG,9606,NaN,NaN,Ki,nM,UO_0000065,NaN,730.0
2,NaN,228175,[],CHEMBL824627,Compound was evaluated for the displacement of...,B,BAO_0000192,BAO_0000219,cell-based format,CC(C(=O)N[C@]1(c2ccccc2)CC[C@@H](N2CCC(c3ccc(F...,...,Homo sapiens,HERG,9606,NaN,NaN,Ki,nM,UO_0000065,NaN,43.0
3,NaN,232020,[],CHEMBL824627,Compound was evaluated for the displacement of...,B,BAO_0000192,BAO_0000219,cell-based format,CC(C(=O)N[C@]1(c2ccccc2)CC[C@H](N2CCCCC2)CC1)c...,...,Homo sapiens,HERG,9606,NaN,NaN,Ki,nM,UO_0000065,NaN,1200.0
4,NaN,232022,[],CHEMBL824627,Compound was evaluated for the displacement of...,B,BAO_0000192,BAO_0000219,cell-based format,CC1CCN([C@H]2CC[C@](NC(=O)C(C)c3cc(C(F)(F)F)cc...,...,Homo sapiens,HERG,9606,NaN,NaN,Ki,nM,UO_0000065,NaN,460.0


In [53]:
df.columns

Index(['activity_comment', 'activity_id', 'activity_properties',
       'assay_chembl_id', 'assay_description', 'assay_type', 'bao_endpoint',
       'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment',
       'data_validity_description', 'document_chembl_id', 'document_journal',
       'document_year', 'ligand_efficiency', 'molecule_chembl_id',
       'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id',
       'standard_flag', 'standard_relation', 'standard_text_value',
       'standard_type', 'standard_units', 'standard_upper_value',
       'standard_value', 'target_chembl_id', 'target_organism',
       'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type',
       'units', 'uo_units', 'upper_value', 'value'],
      dtype='object')

In [54]:
df.activity_comment.value_counts()

Not Active (inhibition < 50% @ 10 uM and thus dose-reponse curve not measured)    835
Not Determined                                                                     99
Not Active                                                                         10
Note: corresponding IC50 reported as Active                                         1
312548                                                                              1
312545                                                                              1
312546                                                                              1
312542                                                                              1
312549                                                                              1
312541                                                                              1
312543                                                                              1
312544                                                

In [55]:
df.assay_type.value_counts()

B    2568
A      46
T       1
Name: assay_type, dtype: int64

In [56]:
df.standard_relation.value_counts()

=    1305
>     364
<       1
Name: standard_relation, dtype: int64

In [57]:
df.units.value_counts()

uM     793
nM     692
/nM      5
Name: units, dtype: int64

In [58]:
import numpy as np

def unitConversion(value, unit='uM'):
    if unit == 'uM':
        value = float(value) * 1000
    elif unit == 'nM' or unit == '\nM':
        value = float(value)
    elif unit == 'microM':
        value = float(value) * 1000000
    else:
        pass
    return value

df.value = [unitConversion(x,df.units[i]) for i,x in enumerate(df.value)]
df.units = 'nM'

In [59]:
df.value = [float(x) for x in df.value]
df = df[df.activity_comment != 'Not Determined']
df = df[df.activity_comment != 'Note: corresponding IC50 reported as Active']
df = df[df.assay_type == 'B'] # 只保留Binding Data

print('共有%s个化合物' %(len(df)))
print('活性化合物(Ki<10uM)有%s个' %len(df[df.value < 10000]))
print('非活性化合物(Ki<10uM)有%s个' %len(df[(df.value >= 10000) | (df.activity_comment == 'Not Active (inhibition < 50% @ 10 uM and thus dose-reponse curve not measured)') | (df.activity_comment == 'Not Active')]))

共有2468个化合物
活性化合物(Ki<10uM)有1246个
非活性化合物(Ki<10uM)有1222个


In [60]:
# activity_comment列标记活性
df.activity_comment[df.value < 10000] = 1
df.activity_comment[(df.value >= 10000) | (df.activity_comment == 'Not Active (inhibition < 50% @ 10 uM and thus dose-reponse curve not measured)') | (df.activity_comment == 'Not Active')] = 0

/home/dahuilangda/Simulation/miniconda3/envs/env_simu/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
df.to_csv('herg_compound_from_chembl_processed.csv')